In [1]:
import meep as mp
import numpy as np

In [2]:
# --- 1. 定义仿真参数 ---

# 几何参数 (单位: 微米, um)
period = 0.5         # 阵列周期
h_substrate = 1.0    # 基底厚度
h_cone = 0.8         # 蛾眼圆锥的高度
w_cone_base = 0.4    # 蛾眼圆锥的底部宽度

# 光学参数
wavelength = 1.0     # 入射光波长 (um)
freq = 1 / wavelength  # 频率

# 材料: 使用复折射率定义金属 (这里用一个典型的近红外金属参数作为例子)
# n_complex = n + ik. MEEP需要定义复介电常数 epsilon = n_complex^2
n_metal = 1.5
k_metal = 7.5
n_complex_metal = n_metal + 1j * k_metal
metal_epsilon = n_complex_metal**2

metal = mp.Medium(epsilon=metal_epsilon)
substrate_medium = mp.Medium(index=1.5) # 假设基底是玻璃

# 仿真区域和分辨率
sx = period
sy = h_substrate + h_cone + 2 * wavelength # 上下留出足够空间
cell = mp.Vector3(sx, sy, 0)
resolution = 50  # pixels/um

AttributeError: module 'meep' has no attribute 'Medium'

In [ ]:

# --- 2. 设置几何结构 ---

geometry = [
    # 基底
    mp.Block(
        center=mp.Vector3(0, -sy/2 + h_substrate/2, 0),
        size=mp.Vector3(mp.inf, h_substrate, mp.inf),
        material=substrate_medium,
    ),
    # 金属蛾眼 (在2D中是一个三角形/锥形)
    mp.Cone(
        center=mp.Vector3(0, h_substrate - sy/2, 0),
        height=h_cone,
        radius=0, # 顶端半径
        radius2=w_cone_base / 2, # 底部半径
        material=metal,
    ),
]

# --- 3. 设置波源 ---

# 在结构上方设置一个平面波源
source_pos_y = 0.5 * sy - wavelength
sources = [
    mp.Source(
        mp.ContinuousSource(frequency=freq),
        component=mp.Ez,  # Ez偏振 (TM波)
        center=mp.Vector3(0, source_pos_y, 0),
        size=mp.Vector3(sx, 0, 0),
    )
]

# --- 4. 设置边界条件 ---

pml_layers = [mp.PML(1.0)] # 1 um 厚的PML吸收边界
boundary_layers = [
    mp.PeriodicBoundary(direction=mp.X),
]

# --- 5. 创建仿真对象 ---

sim = mp.Simulation(
    cell_size=cell,
    boundary_layers=pml_layers,
    # periodic_x=True is an older syntax. Using PeriodicBoundary is modern.
    geometry=geometry,
    sources=sources,
    resolution=resolution,
    extra_boundaries=boundary_layers,
)

# --- 6. 设置通量监视器 ---

# 反射光监视器 (在波源上方)
refl_fr = mp.FluxRegion(
    center=mp.Vector3(0, source_pos_y + 0.5 * wavelength, 0), size=mp.Vector3(sx, 0, 0)
)
refl = sim.add_flux(freq, 0, 1, refl_fr)

# 透射光监视器 (在基底下方)
tran_fr = mp.FluxRegion(
    center=mp.Vector3(0, -0.5 * sy + 0.5, 0), size=mp.Vector3(sx, 0, 0)
)
tran = sim.add_flux(freq, 0, 1, tran_fr)

# --- 7. 运行仿真 ---

# 运行直到场稳定
sim.run(until=200)

# --- 8. 收集数据并进行归一化 ---

# 保存有结构时的反射通量
flux_with_structure = mp.get_fluxes(refl)
reflected_flux = -flux_with_structure[0] # MEEP中通量有方向，反射光是负向
transmitted_flux = mp.get_fluxes(tran)[0]

# --- 重置仿真, 移除结构, 进行归一化运行 ---
sim.reset_meep()

# 定义一个没有几何结构的仿真对象
sim_empty = mp.Simulation(
    cell_size=cell,
    boundary_layers=pml_layers,
    sources=sources,
    resolution=resolution,
    extra_boundaries=boundary_layers,
)

# 在同样位置添加监视器
refl_empty = sim_empty.add_flux(freq, 0, 1, refl_fr)

# 运行空仿真
sim_empty.run(until=200)

# 获取入射光通量
incident_flux = mp.get_fluxes(refl_empty)[0]


# --- 9. 计算并打印结果 ---
reflectance = reflected_flux / incident_flux
transmittance = transmitted_flux / incident_flux
absorbance = 1 - reflectance - transmittance

print(f"--- Results for wavelength = {wavelength} um ---")
print(f"Reflectance (R): {reflectance:.4f}")
print(f"Transmittance (T): {transmittance:.4f}")
print(f"Absorbance (A): {absorbance:.4f}")

# (可选) 输出结构epsilon的图像，以便检查
sim.plot2D(output_plane=mp.Volume(center=mp.Vector3(), size=cell))